Script to label object points.

Adapted from P3PO - [link](https://github.com/mlevy2525/P3PO/blob/main/p3po/data_generation/label_points.ipynb)

In [1]:
import cv2
import pickle
from pathlib import Path

#TODO: Set the task name here -- this will be used to save the output
task_name = "0124_human_drawer_close"
object_name = "objects"

# If the image that shows at the bottom is bgr set original_bgr to True
pickle_path = f"/media/newhdd/projects/tracking_policies/teleop_data/data_franka/processed_data_pkl_aa/{task_name}.pkl"
traj_idx = 0
original_bgr = True

#TODO: If its hard to see the image, you can increase the size_multiplier, this won't affect the selected coordinates
size_multiplier = 1

coordinates_path = f"../../coordinates/{task_name}"

In [2]:
%gui asyncio

import os
from PIL import Image

import numpy as np
from IPython.display import display, Javascript
import ipywidgets as widgets
from ipycanvas import Canvas, hold_canvas
import pickle

import io
import asyncio
import logging

# Define an async function to wait for button click
async def wait_for_click(button):
    # Create a future object
    future = asyncio.Future()
    # Define the click event handler
    def on_button_clicked(b):
        future.set_result(None)
    # Attach the event handler to the button
    button.on_click(on_button_clicked)
    # Wait until the future is set
    await future

class Points():
    def __init__(self, pixel_key, img, coordinates_path, size_multiplier=1):
        logging.getLogger().setLevel(logging.DEBUG)
        logging.info("Starting the Points class")
        self.img = img
        self.size_multiplier = size_multiplier
        self.coordinates_path = coordinates_path
        self.pixel_key = pixel_key

        # Save the image to a bytes buffer
        image = Image.fromarray(self.img)
        size = img.shape
        image = image.resize((size[1] * self.size_multiplier, size[0] * self.size_multiplier))
        buffer = io.BytesIO()
        image.save(buffer, format='PNG')
        buffer.seek(0)

        # Create an IPyWidgets Image widget
        self.canvas = Canvas(width=size[1] * self.size_multiplier, height=size[0] * self.size_multiplier)
        # Define the size of each cell

        self.canvas.put_image_data(np.array(image), 0, 0)

        # Display coordinates
        coords_label = widgets.Label(value="Click on the image to select the coordinates")

        # Define the click event handler
        self.coords = []
        def on_click(x, y):
            coords_label.value = f"Coordinates: ({x}, {y})"
            self.coords.append((0, x, y))

            with hold_canvas(self.canvas):
                self.canvas.put_image_data(np.array(image), 0, 0)  # Redraw the original image

                self.canvas.fill_style = 'red'
                for coord in self.coords:
                    x, y = coord[1] // self.size_multiplier, coord[2] // self.size_multiplier
                    self.canvas.fill_circle(x, y, 2)

        # Connect the click event to the handler
        self.canvas.on_mouse_down(on_click)

        self.button = widgets.Button(description="Save Points")

        # Display the widgets
        self.vbox = widgets.VBox([self.canvas, coords_label, self.button])

        # # Display the widget
        display(self.vbox)

    def on_done(self):
        logging.info("saving")
        Path(self.coordinates_path + "/coords/").mkdir(parents=True, exist_ok=True)
        with open(self.coordinates_path + "/coords/" + f"{self.pixel_key}_{object_name}" + ".pkl", 'wb') as f:
            try:
                pickle.dump(self.coords, f)
            except Exception as e:
                logging.info(e)
        Path(self.coordinates_path + "/images/").mkdir(parents=True, exist_ok=True)
        with open(self.coordinates_path + "/images/" + f"{self.pixel_key}" + ".png", 'wb') as f:
            try:
                image = Image.fromarray(self.img)
                image.save(f)
            except Exception as e:
                logging.info(e)
        logging.info("saved")

In [4]:
# NOTE: Label points for each pixel key. Make sure the order 
# of points is the same across pixel keys.
pixel_key = "pixels1"
    
with open(pickle_path, 'rb') as f:
    data = pickle.load(f)
img = data['observations'][traj_idx][pixel_key][0]
use_video = False
if original_bgr:
    img = img[:,:,::-1]        

async def f():
    point = Points(pixel_key, img, coordinates_path, size_multiplier)
    x = await wait_for_click(point.button)
    point.vbox.close()
    point.canvas.close()
    point.on_done()
asyncio.ensure_future(f())

<Task pending name='Task-6' coro=<f() running at /tmp/ipykernel_1349059/3382082601.py:27>>

INFO:root:Starting the Points class


DEBUG:Comm:handle_msg[d8ae2838ef6c4dcfacf7380572071880]({'header': {'date': datetime.datetime(2025, 1, 24, 18, 51, 37, 934000, tzinfo=tzutc()), 'msg_id': '87cd489f-5e10-4944-9274-9d2995e2604f', 'msg_type': 'comm_msg', 'session': '062ad9ce-42bd-4017-8b0b-0f78d871c6b5', 'username': 'c4007486-b785-437a-99b0-ececdf05e15a', 'version': '5.2'}, 'msg_id': '87cd489f-5e10-4944-9274-9d2995e2604f', 'msg_type': 'comm_msg', 'parent_header': {}, 'metadata': {}, 'content': {'comm_id': 'd8ae2838ef6c4dcfacf7380572071880', 'data': {'method': 'custom', 'content': {'event': 'client_ready'}}}, 'buffers': []})
DEBUG:Comm:handle_msg[d8ae2838ef6c4dcfacf7380572071880]({'header': {'date': datetime.datetime(2025, 1, 24, 18, 51, 38, 485000, tzinfo=tzutc()), 'msg_id': '9af795ac-7553-431b-b024-08200b672f0a', 'msg_type': 'comm_msg', 'session': '062ad9ce-42bd-4017-8b0b-0f78d871c6b5', 'username': 'c4007486-b785-437a-99b0-ececdf05e15a', 'version': '5.2'}, 'msg_id': '9af795ac-7553-431b-b024-08200b672f0a', 'msg_type': 'co